In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn import linear_model, tree, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, accuracy_score
from sklearn import preprocessing
from nltk import pos_tag
import time
from analyze_words import get_final_df
import joblib

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rhedintzaaudryna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from analyze_words import *

In [4]:
def applyModels(model, x_train, y_train):
    print(model)
    model.fit(x_train, y_train)
    return model


def predictModel(model, x_test):
    prediction = model.predict(x_test)
    return prediction

In [5]:
df = get_final_df('test_data/merged_data.csv')

In [7]:
df, idf = df

In [22]:
df.head()

,gross,bloody,burgers,three,heady,beers,blaring,music,ears,still,...,tour very,enjoyable pricing,pricing at,at seems,seems little,steep group,group tour,tour end,end very,Rating
0,2.836342,3.246761,1.821409,2.014717,2.937546,2.638711,2.937546,2.91384,2.879994,1.230529,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.153621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.153621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [9]:
def transformFeatureSelection(model, x):
    return model.transform(x)


def applyFeatureSelection(model, x_train, y_train):
    model = model.fit(x_train, y_train)
    return model

In [10]:
x_train, x_test, y_train, y_test = train_test_split(df.drop("Rating", axis=1),df.Rating,test_size=0.2)
model = linear_model.SGDClassifier()

In [12]:
trained_model = applyModels(model, x_train, y_train)

SGDClassifier()


In [13]:
feature_selection_model = SelectFromModel(trained_model)
trained_feature_selection_model = applyFeatureSelection(feature_selection_model,
                                                        x_train, y_train)
x_train = transformFeatureSelection(trained_feature_selection_model,
                                    x_train)
x_test = transformFeatureSelection(trained_feature_selection_model,
                                   x_test)

In [15]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.89394393, 1.63926851, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.63926851, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
feature_idx = trained_feature_selection_model.get_support()

In [19]:
feature_name = df.drop("Rating", axis=1).columns[feature_idx]

In [64]:
final_model = applyModels(model, x_train, y_train)

SGDClassifier()


ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [70]:
prediction = predictModel(final_model, [x_test[0]])

In [74]:
print(str(prediction))

[3]


In [69]:
x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
np.mean(prediction == y_test)

0.998

In [28]:
import json

In [33]:
with open('columns.json', 'w') as outfile:
    json.dump(list(feature_name), outfile)

In [34]:
with open('idf.json', 'w') as f:
        json.dump(idf, f)

In [35]:
with open("idf.json") as f:
    idf = json.load(f)

In [39]:
with open("columns.json") as f:
    columns = json.load(f)

In [48]:
columns_set = set(columns)

In [50]:
index = pd.Index(columns)

In [51]:
%%time
if 'gross' in index:
    print('yay')

yay
CPU times: user 2.61 ms, sys: 695 µs, total: 3.3 ms
Wall time: 2.87 ms


In [52]:
%%time
if 'gross' in columns_set:
    print('yay')

yay
CPU times: user 195 µs, sys: 187 µs, total: 382 µs
Wall time: 244 µs


In [57]:
%%time
index.get_loc('seems little')

CPU times: user 36 µs, sys: 75 µs, total: 111 µs
Wall time: 114 µs


25173

In [58]:
bla = np.zeros(5)

In [60]:
bla[0] = 1

In [61]:
bla

array([1., 0., 0., 0., 0.])

In [23]:
accuracy_df = pd.DataFrame({'prediction': prediction, 'test': y_test})

In [24]:
accuracy_df[accuracy_df.prediction != accuracy_df.test]

,prediction,test
5292,2,3
9039,2,5
7904,5,4
4306,2,3
127,3,1
...,...,...
6284,5,4
4108,2,3
5641,4,3
9640,4,5


In [28]:
def evaluateModel(model, prediction, x, y):
    start_time = time.time()
    confusion = confusion_matrix(y.to_numpy(), prediction)
    accuracy = accuracy_score(y, prediction)*100
    f1 = f1_score(y, prediction, average='macro')
    cr = classification_report(prediction, y)
    completed_time = time.time() - start_time
    print('Evaluation Time: ', completed_time)
    return {
        'prediction': prediction,
        'confusion': confusion,
        'accuracy': accuracy,
        'classification report': cr,
        'f1': f1
    }

In [45]:
trained_model.get_features()

AttributeError: 'SGDClassifier' object has no attribute 'get_features'

In [33]:
print(accuracy_score(y_test, prediction)*100)

57.74747474747475


In [34]:
confusion_matrix(y_test, prediction)

array([[1449,  417,  110,    7,    0],
       [ 237, 1575,    0,   13,  153],
       [ 213,  326, 1026,  360,   55],
       [ 166,  225,  146,  754,  691],
       [ 196,  253,  118,  497,  913]])

In [31]:
print(classification_report(prediction, y_test))

              precision    recall  f1-score   support

           1       0.73      0.64      0.68      2261
           2       0.80      0.56      0.66      2796
           3       0.52      0.73      0.61      1400
           4       0.38      0.46      0.42      1631
           5       0.46      0.50      0.48      1812

    accuracy                           0.58      9900
   macro avg       0.58      0.58      0.57      9900
weighted avg       0.61      0.58      0.59      9900



In [ ]:
#  specify the inputs: ngrams, removing stop words, removing punctuations or not 
#  If we have 100,000 columns, forward  selection to choose only the best columns
#  specifiy model parameters like loss, alpha, etc. 

In [ ]:
# ngrams = [1,2,3,4,5]
# stop_words = [0, 10, 20]
# lemmatize = [True, False]

# alpha = [0.0001, 0.001, 0.01, 0.1, 1]




#  10 columns 
# 9 columns 
# 1 column used (40%, 2 columns used 8 columns left (50%, 3 columns (49%)
best_comb = {} # include ngram, stop words, lemmatize, idf, alpha
best_accuracy = -1
for ngram in ngrams: 
    for stop_word in stop_words:
        for lemmatize1 in lemmatize: 
            for alpha1 in alpha:
                df = get_final_df()
                model = linear_model.SGDClassifier(alpha=alpha1)
                

return best_comb, best_df, idf



#forward selection

#save the model
return 
                
                




In [42]:
def forward_selection(df):
    '''
    Given a dataset with P predictor variables, uses forward selection to
    select models for every value of K between 1 and P.

    Inputs:
        dataset: (DataSet object) a dataset

    Returns:
        A list (of length P) of Model objects. The first element is the
        model where K=1, the second element is the model where K=2, and so on.
    '''

    models = []
    pred_vars_avail = dataset.pred_vars[:]
    pred_vars_used = []
    num_vars = len(dataset.pred_vars)

    for _ in range(num_vars):
        max_R2 = 0
        max_model = None
        max_variable = None
        for var in pred_vars_avail:
            model = Model(dataset, pred_vars_used + [var])
            if model.R2 > max_R2:
                max_model = model
                max_R2 = model.R2
                max_variable = var
        pred_vars_used.append(max_variable)
        pred_vars_avail.remove(max_variable)
        models.append(max_model)

    return models

In [35]:
def get_stop_words(all_tokens, num_stop_words=20):
    '''
    Obtain the particular stop words (most frequently occurring words) in the
    sample, which may differ from a list of generic stop words.

    Inputs:
        - all_tokens (list of list of strings): a list of list of all tokens
        - num_stop_words (int): number of stop words to remove

    Returns:
        - list of most common words
    '''
    all_words = list(itertools.chain.from_iterable(all_tokens))
    freq_dist = FreqDist(all_words)
    stop_words = freq_dist.most_common(num_stop_words)

    return [word[0] for word in stop_words]

In [38]:
df = pd.read_csv('test_data/merged_data.csv', usecols=[0, 1], names=[
                     'Rating', 'Text'], header=None)
all_tokens = [processing(text) for text in df.Text]

In [40]:
store = get_stop_words(all_tokens, 100)

In [41]:
store[:20]

['the',
 'and',
 'i',
 'a',
 'to',
 'was',
 'it',
 'of',
 'we',
 'for',
 'in',
 'is',
 'but',
 'that',
 'this',
 'with',
 'they',
 'were',
 'my',
 'not']

In [75]:
run main.py

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rhedintzaaudryna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


   Welcome to the Suggested Star Rating System!

            Copy and paste your review.

      Press Control-D to exit or type quit()

Enter review here: blabla
Please input a longer review.
Enter review here: hello my name is audry. i'm an econ major studying at the university of chicago.
Your suggest star rating is: 5
Thank you for using our Suggested Star Rating System!


NameError: name 'quit' is not defined